<a href="https://colab.research.google.com/github/cinquecentoandrey/transformers/blob/main/%D0%9F%D1%80%D0%B8%D0%BC%D0%B5%D1%80%D1%8B_%D0%A2%D1%80%D0%B0%D0%BD%D1%81%D1%84%D0%BE%D1%80%D0%BC%D0%B5%D1%80%D1%8B%2BGradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers gradio

In [ ]:
from transformers import pipeline
import gradio as gr
from collections import OrderedDict

# **Пример 1. Анализ тональности текста**

In [ ]:
classifier = pipeline(
    "sentiment-analysis",  # Тип задачи - анализ тональности
    model="cointegrated/rubert-tiny2-cedr-emotion-detection",  # Модель для русского языка
    tokenizer="cointegrated/rubert-tiny2-cedr-emotion-detection"  # Соответствующий токенизатор
)

# Создаем словарь для перевода эмоций на русский
emotion_map = OrderedDict([
    ('neutral', "Нейтральный"),
    ('joy', "Радость"),
    ('sadness', "Грусть"),
    ('surprise', "Удивление"),
    ('fear', "Страх"),
    ('anger', "Злость"),
    ('no_emotion', "Безразлично")
])

# Основная функция для анализа текста
def analyze_sentiment(text):
    if not text.strip():
        return "Пожалуйста, введите текст для анализа"

    try:
        # Ограничиваем длину текста для стабильности работы модели
        text = text[:512]

        # Получаем результат от модели (возвращает список, берем первый элемент)
        result = classifier(text)[0]

        # Преобразуем метку эмоции на русский
        emotion_label = result['label']
        emotion_ru = emotion_map.get(emotion_label, emotion_label)
        score = result['score']  # Уверенность модели

        # Создаем визуальный прогресс-бар из emoji
        filled_squares = int(score * 10)  # Переводим вероятность в 10-балльную шкалу
        progress_bar = "🟩" * filled_squares + "⬜" * (10 - filled_squares)

        return f"""
        <div style='font-family: Arial; border-left: 4px solid #000000 ; padding-left: 12px;'>
            <div style='font-size: 20px; font-weight: bold; color: #000000 ; margin-bottom: 8px;'>
                {emotion_ru}
            </div>
            <div style='font-size: 16px; margin-bottom: 12px;'>
                Уверенность: {score:.0%}
            </div>
            <div style='font-size: 24px; letter-spacing: 2px;'>
                {progress_bar}
            </div>
        </div>
        """
    except Exception as e:
        return f"<div style='color: #FF5722;'>Произошла ошибка: {str(e)}</div>"

# Создаем интерфейс
iface = gr.Interface(
    fn=analyze_sentiment,  # Функция для обработки ввода

    # Настраиваем поле ввода текста
    inputs=gr.Textbox(
        lines=5,  # Высота текстового поля (5 строк)
        placeholder="Введите текст...",  # Подсказка внутри поля
        label="Текст для анализа"  # Надпись над полем ввода
    ),

    # Настраиваем вывод (поддерживаем HTML-разметку)
    outputs=gr.HTML(label="Результат анализа тональности"),

    # Заголовок интерфейса
    title="Анализатор тональности текста",

    # Примеры для быстрого тестирования
    examples=[
        ["Это просто потрясающе!"],
        ["Неуд за сессию это не катастрофа."],
        ["Сегодня обычный день."],
        ["Технологии искусственного интеллекта развиваются стремительно."]
    ],

    # Отключаем кнопку "Flag" (не используем систему отчетов)
    allow_flagging="never"
)

# Запускаем веб-интерфейс
iface.launch()

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://492761f4a006a2d024.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Пример 2. Генерация текста**
В данном задание можно поэкспериментировать с настройками дополнительных параметров - температурой, кол-вом beam'ов, top-p (разнообразием), чтобы оценить как меняется сгенерированный текст.

In [ ]:
generator = pipeline(
    "text-generation",
    model="sberbank-ai/rugpt3large_based_on_gpt2",
    device="cpu"
)

def generate_text(prompt, max_length, temperature, top_p, num_beams):
    try:
        results = generator(
            prompt,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            num_beams=num_beams,
            do_sample=True,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

        generated_text = results[0]["generated_text"]

        highlighted_text = generated_text.replace(
            prompt,
            f"<span style='color: #4CAF50; font-weight: bold;'>{prompt}</span>"
        ) if prompt else generated_text

        return f"""
        <div style='font-family: Arial; line-height: 1.6; border-left: 4px solid #4CAF50; padding-left: 15px;'>
            {highlighted_text}
        </div>
        """
    except Exception as e:
        return f"<div style='color: red;'>Ошибка генерации: {str(e)}</div>"

# Создаем интерфейс
iface = gr.Interface(
    # Основная функция обработки (должна принимать столько аргументов, сколько элементов в inputs)
    fn=generate_text,

    # Список входных элементов интерфейса
    inputs=[
        # Текстовое поле для промпта
        gr.Textbox(
            lines=3,  # Начальная высота в 3 строки
            placeholder="Введите начало текста или тему для генерации...",  # Подсказка в пустом поле
            label="Промпт",  # Надпись над полем ввода
            max_lines=5  # Максимальное количество строк при расширении
        ),

        # Слайдер для контроля длины текста
        gr.Slider(
            minimum=20,  # Минимальное значение
            maximum=300,  # Максимальное значение
            value=100,  # Значение по умолчанию
            step=10,  # Шаг изменения
            label="Максимальная длина"  # Подпись слайдера
        ),

        # Слайдер для температуры (контроль случайности)
        gr.Slider(
            0.1, 1.5,  # Диапазон значений
            value=0.9,  # Значение по умолчанию
            step=0.1,  # Шаг изменения
            label="Температура (креативность)"  # Подпись
        ),

        # Слайдер для top-p фильтрации
        gr.Slider(
            0.5, 1.0,  # Диапазон
            value=0.95,  # По умолчанию
            step=0.05,  # Шаг
            label="Top-p (разнообразие)"  # Подпись
        ),

        # Выпадающий список для выбора количества beam'ов
        gr.Dropdown(
            choices=[1, 2, 3, 4, 5],  # Доступные варианты
            value=3,  # Значение по умолчанию
            label="Количество beam'ов"  # Подпись
        )
    ],

    # Настройка вывода (с поддержкой HTML)
    outputs=gr.HTML(
        label="Сгенерированный текст"  # Подпись поля вывода
    ),

    # Заголовок интерфейса
    title="Генератор текста",

    # Описание с HTML-разметкой
    description="""
    <div style='margin-bottom: 15px;'>
        Генерация текста на русском языке с помощью модели RuGPT-3 Large.<br>
        Поэкспериментируйте с параметрами для разных результатов!
    </div>
    """,

    # Примеры для быстрого тестирования
    examples=[
        # Каждый пример - список значений для всех input элементов
        ["В далеком будущем, когда человечество колонизировало Марс", 150, 0.7, 0.9, 3],
        ["Философский вопрос: может ли искусственный интеллект", 200, 0.8, 0.95, 4],
        ["Рецепт идеального борща:", 100, 0.5, 0.85, 2]
    ],

    # Настройка темы интерфейса
    theme=gr.themes.Default(
        primary_hue="green",  # Основной цвет (кнопки, акценты)
        secondary_hue="teal"  # Дополнительный цвет
    ),

    # Отключаем кнопку репортов
    allow_flagging="never"
)

# Запускаем интерфейс
iface.launch()

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3d93b2e5a2c8518ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Пример 3. Перевод текста**


In [ ]:
MODELS = {
    "Английский → Русский": "Helsinki-NLP/opus-mt-en-ru",
    "Русский → Английский": "Helsinki-NLP/opus-mt-ru-en",
}

loaded_models = {}

def translate(text, direction):
    if not text.strip():
        return "Введите текст для перевода"

    try:
        if direction not in loaded_models:
            loaded_models[direction] = pipeline(
                "translation",
                model=MODELS[direction]
            )

        result = loaded_models[direction](text)[0]['translation_text']
        return result

    except Exception as e:
        return f"Ошибка перевода: {str(e)}"

# Создаем интерфейс переводчика
iface = gr.Interface(
    # Основная функция, которая будет обрабатывать перевод
    fn=translate,  # Функция должна принимать текст и направление перевода

    # Входные элементы интерфейса
    inputs=[
        # Текстовое поле для ввода исходного текста
        gr.Textbox(
            lines=3,  # Высота текстового поля (3 строки)
            placeholder="Введите текст для перевода...",  # Подсказка в пустом поле
            label="Исходный текст"  # Надпись над полем ввода
        ),

        # Выпадающий список для выбора направления перевода
        gr.Dropdown(
            choices=list(MODELS.keys()),  # Варианты из словаря MODELS
            value="Английский → Русский",  # Значение по умолчанию
            label="Направление перевода"  # Подпись элемента
        )
    ],

    # Выходное поле для перевода
    outputs=gr.Textbox(
        label="Перевод",  # Надпись над полем вывода
        lines=5  # Высота поля вывода (5 строк)
    ),

    # Заголовок интерфейса
    title="Переводчик",

    # Примеры для быстрого тестирования
    examples=[
        # Первый пример - английский текст
        ["Hello world", "Английский → Русский"],
        # Второй пример - русский текст
        ["Привет мир", "Русский → Английский"]
    ],

    # Отключаем кнопку "пожаловаться" (не используем систему репортов)
    allow_flagging="never",

    # Настраиваем цветовую тему интерфейса
    theme=gr.themes.Default(
        primary_hue="blue",  # Основной синий цвет для кнопок и акцентов
        secondary_hue="gray"  # Вторичный серый цвет
    )
)

# Запускаем веб-интерфейс
iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://403ca8e9963df720e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Пример 4. Заполнение пропусков**

In [ ]:
MODELS = {
    "Английский (BERT)": {
        "name": "bert-base-uncased",
        "pipeline": None,
        "mask_token": "[MASK]"
    },
    "Русский (RuBERT)": {
        "name": "DeepPavlov/rubert-base-cased",
        "pipeline": None,
        "mask_token": "[MASK]"
    },
    "Мультиязычный (XLM)": {
        "name": "xlm-roberta-base",
        "pipeline": None,
        "mask_token": "<mask>"
    }
}

for model_name, config in MODELS.items():
    try:
        MODELS[model_name]["pipeline"] = pipeline(
            "fill-mask",
            model=config["name"]
        )
        print(f"{model_name} загружена")
    except Exception as e:
        print(f"Ошибка загрузки {model_name}: {str(e)}")

def predict_masked(text, model_choice):
    if not text.strip():
        return "Введите текст с пропуском"

    config = MODELS.get(model_choice)
    if not config or not config["pipeline"]:
        return "Модель не загружена"

    mask = config["mask_token"]
    if mask not in text:
        return f"Вставьте '{mask}' в текст"

    try:
        results = config["pipeline"](text, top_k=5)
        output = []
        for i, res in enumerate(results, 1):
            seq = res['sequence'].replace("[CLS]", "").replace("[SEP]", "").strip()
            output.append(f"{i}. {seq} (вероятность: {res['score']:.3f})")
        return "\n".join(output)
    except Exception as e:
        return f"Ошибка: {str(e)}"

examples = [
    ["The capital of France is [MASK].", "Английский (BERT)"],
    ["Столица России - [MASK].", "Русский (RuBERT)"],
    ["Paris is the <mask> of France.", "Мультиязычный (XLM)"]
]

# Создаем интерфейс
iface = gr.Interface(
    # Основная функция обработки (принимает текст и выбранную модель)
    fn=predict_masked,

    # Входные элементы интерфейса
    inputs=[
        # Текстовое поле для ввода с пропуском
        gr.Textbox(
            lines=2,  # Высота в 2 строки
            placeholder="Введите текст с [MASK] или <mask>...",  # Подсказка
            label="Текст с пропуском"  # Надпись над полем
        ),

        # Выпадающий список для выбора модели
        gr.Dropdown(
            list(MODELS.keys()),  # Варианты из словаря MODELS
            value="Английский (BERT)",  # Модель по умолчанию
            label="Выберите модель"  # Подпись элемента
        )
    ],

    # Выходное поле с результатами
    outputs=gr.Textbox(
        label="Топ-5 вариантов",  # Заголовок поля вывода
        lines=6  # Высота поля (6 строк)
    ),

    # Заголовок интерфейса
    title="🔄 Универсальный заполнитель пропусков",

    # Описание под заголовком
    description="Предзагруженные модели для английского, русского и мультиязычных текстов",

    # Примеры для быстрого тестирования
    examples=examples,

    # Настройка цветовой темы
    theme=gr.themes.Soft(  # "Мягкая" тема
        primary_hue="blue",  # Основной синий цвет
        secondary_hue="gray"  # Вторичный серый цвет
    )
)

# Запускаем веб-интерфейс
iface.launch()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Английский (BERT) загружена


Device set to use cpu


Русский (RuBERT) загружена


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Мультиязычный (XLM) загружена
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c9e3f12d637116b447.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
